In [1]:
!tree

卷 Data 的文件夹 PATH 列表
卷序列号为 AC87-A4BD
D:.
├─.idea
│  └─inspectionProfiles
└─archive
    ├─training
    │  └─training
    │      ├─n0
    │      ├─n1
    │      ├─n2
    │      ├─n3
    │      ├─n4
    │      ├─n5
    │      ├─n6
    │      ├─n7
    │      ├─n8
    │      └─n9
    └─validation
        └─validation
            ├─n0
            ├─n1
            ├─n2
            ├─n3
            ├─n4
            ├─n5
            ├─n6
            ├─n7
            ├─n8
            └─n9


In [2]:
"""
10 Monkey 分类 - 深度可分离卷积实现
使用 MobileNetV2（基于深度可分离卷积的轻量级网络）
"""

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

# ==================== 配置 ====================
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32
NUM_EPOCHS = 30
NUM_CLASSES = 10
LEARNING_RATE = 0.001

# 数据路径（根据你的目录结构调整）
DATA_DIR = 'archive'
TRAIN_DIR = os.path.join(DATA_DIR, 'training', 'training')
VAL_DIR = os.path.join(DATA_DIR, 'validation', 'validation')

# ==================== 数据增强 ====================
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ==================== 加载数据 ====================
train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=train_transform)
val_dataset = datasets.ImageFolder(VAL_DIR, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print(f"训练集: {len(train_dataset)} 张, 验证集: {len(val_dataset)} 张")
print(f"类别: {train_dataset.classes}")

# ==================== 模型 - MobileNetV2（深度可分离卷积）====================
def create_mobilenet_v2(num_classes=10):
    """加载预训练的MobileNetV2，修改分类头"""
    model = models.mobilenet_v2(pretrained=True)

    # 冻结部分层（可选）
    for param in model.features[:10].parameters():
        param.requires_grad = False

    # 修改分类器
    model.classifier = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(model.last_channel, num_classes)
    )

    return model

model = create_mobilenet_v2(NUM_CLASSES).to(DEVICE)

# ==================== 损失函数和优化器 ====================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                       lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                  factor=0.5, patience=3)

# ==================== 训练函数 ====================
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    return running_loss / len(loader), 100. * correct / total

def validate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    return running_loss / len(loader), 100. * correct / total

# ==================== 训练循环 ====================
print(f"\n使用设备: {DEVICE}")
print("开始训练（深度可分离卷积 - MobileNetV2）...\n")

best_acc = 0.0
for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)

    scheduler.step(val_loss)

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'best_mobilenet.pth')

    print(f"Epoch {epoch+1:02d}/{NUM_EPOCHS} | "
          f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")

print(f"\n✓ 训练完成！最佳验证准确率: {best_acc:.2f}%")

训练集: 1097 张, 验证集: 272 张
类别: ['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9']


C:\Users\16416\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\16416\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\16416/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:05<00:00, 2.69MB/s]



使用设备: cuda
开始训练（深度可分离卷积 - MobileNetV2）...

Epoch 01/30 | Train Loss: 0.8756, Acc: 71.65% | Val Loss: 0.3590, Acc: 88.97%
Epoch 02/30 | Train Loss: 0.5866, Acc: 80.95% | Val Loss: 0.3255, Acc: 90.81%
Epoch 03/30 | Train Loss: 0.4511, Acc: 85.78% | Val Loss: 0.2262, Acc: 92.28%
Epoch 04/30 | Train Loss: 0.4680, Acc: 84.59% | Val Loss: 0.2103, Acc: 93.01%
Epoch 05/30 | Train Loss: 0.4365, Acc: 86.24% | Val Loss: 0.1735, Acc: 95.96%
Epoch 06/30 | Train Loss: 0.3656, Acc: 88.24% | Val Loss: 0.1256, Acc: 94.85%
Epoch 07/30 | Train Loss: 0.5206, Acc: 84.87% | Val Loss: 0.1323, Acc: 96.32%
Epoch 08/30 | Train Loss: 0.4232, Acc: 86.51% | Val Loss: 0.1669, Acc: 95.22%
Epoch 09/30 | Train Loss: 0.4439, Acc: 85.60% | Val Loss: 0.2006, Acc: 94.12%
Epoch 10/30 | Train Loss: 0.3500, Acc: 88.15% | Val Loss: 0.0733, Acc: 97.43%
Epoch 11/30 | Train Loss: 0.2781, Acc: 91.07% | Val Loss: 0.1185, Acc: 96.32%
Epoch 12/30 | Train Loss: 0.2482, Acc: 92.16% | Val Loss: 0.1023, Acc: 96.32%
Epoch 13/30 | Train 